In [56]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [57]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [58]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [59]:
model_tag = 'cohere'  
dataset_tag = 'social_media'
prompt = 'emb'
temp = 0
number_of_experimet = 1
selected_samples =  'aggl'

In [60]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [61]:
dataset_for_annotation.columns

Index(['text', 'label'], dtype='object')

In [62]:
print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

CohereLLM
Params: {}                                                    text           label
0     2020 is in less than 2 hours and I don't feel ...        moderate
1     Depression or laziness? : For quite a long tim...        moderate
2     $1,000,000 or no more depression? : Which woul...  not depression
3     Negative Nancy : I’ve recently started dating ...        moderate
4     I’m sure I’ll die alone at this point : I know...        moderate
...                                                 ...             ...
1062  Two labels, two boxes : A month or so ago, in ...        moderate
1063  I almost cried while my family was wishing eac...        moderate
1064  Am I the only one who hates new years? : Now t...        moderate
1065  I just want to smile. : I can’t smile. It does...          severe
1066  new year new chance : I wanted to go on a new ...        moderate

[1067 rows x 2 columns] Task: Read the following text and determine the most appropriate category based on its con

In [63]:
text_col = "text"
label_col = "label"
output_col = "output"

In [64]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation,
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [65]:
df = annotator.get_results(is_save=True, output_path=output_path)

Nr: 0 Predicted label: moderate
Nr: 1 Predicted label: moderate
Nr: 2 Predicted label: not depression
Nr: 3 Predicted label: moderate
Nr: 4 Predicted label: severe
Nr: 5 Predicted label: moderate
Nr: 6 Predicted label: moderate
Nr: 7 Predicted label: not depression
Nr: 8 Predicted label: moderate
Nr: 9 Predicted label: moderate
Nr: 10 Predicted label: moderate
Nr: 11 Predicted label: severe
Nr: 12 Predicted label: moderate
Nr: 13 Predicted label: not depression
Nr: 14 Predicted label: moderate
Nr: 15 Predicted label: not depression
Nr: 16 Predicted label: moderate
Nr: 17 Predicted label: moderate
Nr: 18 Predicted label: moderate
Nr: 19 Predicted label: not depression
Nr: 20 Predicted label: moderate
Nr: 21 Predicted label: severe
Nr: 22 Predicted label: not depression
Nr: 23 Predicted label: not depression
Nr: 24 Predicted label: moderate
Nr: 25 Predicted label: moderate
Nr: 26 Predicted label: severe
Nr: 27 Predicted label: moderate
Nr: 28 Predicted label: moderate
Nr: 29 Predicted la

In [66]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/emb_temp0/cohere/social_media_cohere_emb_aggl_temp0_exp1.csv


In [67]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label
0,2020 is in less than 2 hours and I don't feel ...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate
1,Depression or laziness? : For quite a long tim...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate
2,"$1,000,000 or no more depression? : Which woul...",not depression,"{'content': [{'token': 'not', 'bytes': [110, 1...","[{'token': 'not', 'bytes': [110, 111, 116], 'l...",not depression
3,Negative Nancy : I’ve recently started dating ...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate
4,I’m sure I’ll die alone at this point : I know...,severe,"{'content': [{'token': 'severe', 'bytes': [115...","[{'token': 'severe', 'bytes': [115, 101, 118, ...",moderate
...,...,...,...,...,...
1062,"Two labels, two boxes : A month or so ago, in ...",not depression,"{'content': [{'token': 'not', 'bytes': [110, 1...","[{'token': 'not', 'bytes': [110, 111, 116], 'l...",moderate
1063,I almost cried while my family was wishing eac...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate
1064,Am I the only one who hates new years? : Now t...,not depression,"{'content': [{'token': 'not', 'bytes': [110, 1...","[{'token': 'not', 'bytes': [110, 111, 116], 'l...",moderate
1065,I just want to smile. : I can’t smile. It does...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",severe


In [68]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_first_bracketed(text):
        match = re.search(r'\[([^\[\]]+?)\]', str(text))
        return match.group(1) if match else None
    
    df[write_col] = df[read_col].apply(extract_first_bracketed)
    return df


In [69]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, output_col, 'extracted_label')
    df.to_csv(path, index=False)
    
    return df


In [70]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,2020 is in less than 2 hours and I don't feel ...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate,True,None
1,Depression or laziness? : For quite a long tim...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate,True,None
2,"$1,000,000 or no more depression? : Which woul...",not depression,"{'content': [{'token': 'not', 'bytes': [110, 1...","[{'token': 'not', 'bytes': [110, 111, 116], 'l...",not depression,True,None
3,Negative Nancy : I’ve recently started dating ...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate,True,None
4,I’m sure I’ll die alone at this point : I know...,severe,"{'content': [{'token': 'severe', 'bytes': [115...","[{'token': 'severe', 'bytes': [115, 101, 118, ...",moderate,True,None
...,...,...,...,...,...,...,...
1062,"Two labels, two boxes : A month or so ago, in ...",not depression,"{'content': [{'token': 'not', 'bytes': [110, 1...","[{'token': 'not', 'bytes': [110, 111, 116], 'l...",moderate,False,None
1063,I almost cried while my family was wishing eac...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",moderate,False,None
1064,Am I the only one who hates new years? : Now t...,not depression,"{'content': [{'token': 'not', 'bytes': [110, 1...","[{'token': 'not', 'bytes': [110, 111, 116], 'l...",moderate,False,None
1065,I just want to smile. : I can’t smile. It does...,moderate,"{'content': [{'token': 'moderate', 'bytes': [1...","[{'token': 'moderate', 'bytes': [109, 111, 100...",severe,False,None
